# Preamble

This notebook will train a very simple model that will compare two numbers and tell us if one is larger than the other.

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import InputLayer
from tensorflow.data import Dataset
import numpy as np
import itertools

# Create a dataset to train model

In [10]:
def data_generator():
    while(True):
        number1 = np.random.uniform();
        number2 = np.random.uniform();
        # Our input data is an array containing 2 numbers
        X = [number1, number2]
        # Our label is 1 or 0
        Y = 1 if number2 > number1 else 0
        # Our generator should return the input data and the label
        yield X, [Y]
        
# Create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)
train_dataset = train_dataset.batch(batch_size=50)

# Define model

Model consist of input and output layers.
The activation function for output is sigmoid. It will output a value between `0` and `1`.

In [11]:
model = Sequential([
    InputLayer(input_shape=(2)),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile model

For our loss function we need to use `BinaryCrossentropy`.
Crossentropy quantifies the difference between two probability distribution.
We have a binary distribution (`True` or `False`) so we use binary crossentropy to compare the output from our model with the true distribution.

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 5)                 15        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(
    train_dataset,
    steps_per_epoch=1000,
    epochs=5
)

Epoch 1/5
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4937 - accuracy: 0.9563
Epoch 2/5
1000/1000 [==============================] - 10s 10ms/step - loss: 0.2092 - accuracy: 0.9949
Epoch 3/5
1000/1000 [==============================] - 10s 10ms/step - loss: 0.1292 - accuracy: 0.9964
Epoch 4/5
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0970 - accuracy: 0.9977
Epoch 5/5
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0779 - accuracy: 0.9972


# Testing model

We are going to feed in some values and see what our model predicts.

In [14]:
test_x = np.array([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.1],
    [0.7, 0.2]
])
y = model.predict_on_batch(test_x)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(y)

[[0.91]
 [0.91]
 [0.00]
 [0.00]]


# Export model

Before use model we need to convert model into a tflite model

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]

converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

tflite_quant_model = converter.convert()
open("model.tflite", "wb").write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmppsh9q1rp/assets


INFO:tensorflow:Assets written to: /tmp/tmppsh9q1rp/assets


2024

Now we can convert model to the C code:
```
$ xxd -i model.tflite > model.cc
```